In [1]:
from sklearn.datasets import make_classification
import torch

In [2]:
X,y=make_classification(
    n_samples=10,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_repeated=0,
    n_classes=2,
    random_state=42,

)

In [3]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [4]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [5]:
X.shape

(10, 2)

In [6]:
X=torch.tensor(X,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.long)

In [7]:
from torch.utils.data import DataLoader, Dataset

In [8]:
X,y

(tensor([[ 1.0683, -0.9701],
         [-1.1402, -0.8388],
         [-2.8954,  1.9769],
         [-0.7206, -0.9606],
         [-1.9629, -0.9923],
         [-0.9382, -0.5430],
         [ 1.7273, -1.1858],
         [ 1.7774,  1.5116],
         [ 1.8997,  0.8344],
         [-0.5872, -1.9717]]),
 tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0]))

In [9]:
X.shape

torch.Size([10, 2])

In [10]:
class Customdataset(Dataset):
    def __init__(self,features,lables):
        self.features = features
        self.lables = lables

    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.lables[idx]
        return feature, label
        

In [11]:
dataset=Customdataset(X,y)
# Define the model

len(dataset)

10

In [12]:
dataset[2]

(tensor([-2.8954,  1.9769]), tensor(0))

In [13]:
dataset[8]                                                                                                                         

(tensor([1.8997, 0.8344]), tensor(1))

In [14]:
dataloader= DataLoader(dataset,batch_size=2,shuffle=True)



In [15]:
len(dataset)

10

In [16]:
y

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [17]:
import torch.nn as nn

In [18]:
class ANN(nn.Module):
    
    def __init__(self,num_features):

        super().__init__()
        
        self.model=nn.Sequential(
            nn.Linear(num_features,5),
            nn.ReLU(),
            # --- CHANGE THIS LINE ---
            nn.Linear(5, 2)  # Was nn.Linear(5, 1)
        )

    def forward(self,x):
        return self.model(x)

In [ ]:
# Cell 143
lr=0.01
epochs=10

model=ANN(X.shape[1])
# --- CHANGE THIS LINE ---
loss_fn = nn.BCEWithLogitsLoss() # Was nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
# Cell 144
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in dataloader:

    # forward pass
    outputs = model(batch_features) # Shape [2, 1]

    # --- MODIFY THIS LINE ---
    # Reshape labels from [2] to [2, 1] and convert to float
    loss = loss_fn(outputs, batch_labels.unsqueeze(1).float())

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(dataloader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

ValueError: Target size (torch.Size([2, 1])) must be the same as input size (torch.Size([2, 2]))